# ExGI - Gene Interaction Extraction algorithm

 The following two cell imports the needed packages, pretrained BioBERT weights and loads pretrained BioBert model

In [ ]:
!pip install datasets

!wget http://nlp.dmis.korea.edu/projects/biobert-2020-checkpoints/biobert_v1.1_pubmed.tar.gz
!tar -xvzf /content/biobert_v1.1_pubmed.tar.gz

!pip install pytorch_transformers
!pip install transformers
!tar -xzf biobert_weights
!ls biobert_v1.1_pubmed/
!transformers-cli convert --model_type bert --tf_checkpoint biobert_v1.1_pubmed/model.ckpt-1000000 --config biobert_v1.1_pubmed/bert_config.json --pytorch_dump_output biobert_v1.1_pubmed/pytorch_model.bin
!ls biobert_v1.1_pubmed/
!mv biobert_v1.1_pubmed/bert_config.json biobert_v1.1_pubmed/config.json
!ls biobert_v1.1_pubmed/
from pytorch_transformers import BertModel
DATA_DIR="."
import os
import numpy as np
import pickle
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# install BERT
!pip install pytorch_pretrained_bert pytorch-nlp

# BERT imports
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
!pip install Keras-Preprocessing
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
#% matplotlib inline

# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)
model = BertForSequenceClassification.from_pretrained("biobert_v1.1_pubmed", num_labels=2)#binary classification
model.cuda()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.4 MB/s eta 0:00:00
--2023-07-30 15:04:43--  http://nlp.dmis.korea.edu/projects/biobert-2020-checkpoints/biobert_v1.1_pubmed.tar.gz
Resolving nlp.dmis.korea.edu (nlp.dmis.korea.edu)... 163.152.163.168
Connecting to nlp.dmis.korea.edu (nlp.dmis.korea.edu)|163.152.163.168|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 401403346 (383M) [application/x-gzip]
Saving to: ‘biobert_v1.1_pubmed.tar.gz’

biobert_v1.1_pubmed 100%[===================>] 382.81M  3.32MB/s    in 2m 0s   

2023-07-30 15:06:45 (3.18 MB/s) - ‘biobert_v1.1_pubmed.tar.gz’ saved [401403346/401403346]

biobert_v1.1_pubmed

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=Fa

In [ ]:
import numpy as np
import pandas as pd
import bs4
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
from openpyxl import load_workbook
!pip install biopython
!pip install Bio

from Bio import Entrez
import nltk.data
import nltk
nltk.download('punkt')
import requests
import itertools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 5.5 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# **Pre-processing - Search query with target specific keywords and retrieve relevant abstracts:** The following cell contains the methods to fetch literature abstracts as per selected keywords


In [ ]:
#Abstract Fetch
def search(query,amt):
    Entrez.email = 'jassig2905@GMAIL.COM'
    handle = Entrez.esearch(db='pubmed',
                            sort='relevance',
                            retmax=amt,
                            retmode='xml',
                            term=query)
    results = Entrez.read(handle)
    return results

def make_a_link(id):
    return("https://pubmed.ncbi.nlm.nih.gov/"+str(id)+"/")

def extract_articles(list_of_ids):
    Article_title = []
    Article_Abstract = []
    Pub_med_link = []
    Link_full_articles = []
    Type_of_articles = []
    for id in list_of_ids:
        uClient  = uReq(make_a_link(id))
        page_html = uClient.read()
        page_soup = soup(page_html, "html.parser")
        Title = page_soup.find("h1",{"class":"heading-title"}).text.replace("\n","").strip()

        Abstract = page_soup.find("div",{"class":"abstract-content"})
        if(Abstract is not None):
            Abstract = Abstract.text.replace("\n","").strip()
        #print(page_soup.find("div",{'class':'action-buttons inline'}))
        Full_Article_link = page_soup.find("a",{"data-ga-action":"DOI"})
        if(Full_Article_link is not None):
            Full_Article_link = page_soup.find("a",{"data-ga-action":"DOI"})["href"]
            Link_full_articles.append(Full_Article_link)
        else:
            Link_full_articles.append("NA")
        Type = page_soup.find("span","article-source").text.strip()
        Article_title.append(Title)
        Article_Abstract.append(Abstract)
        Pub_med_link.append(make_a_link(id))
        #Link_full_articles.append(Full_Article_link)
        Type_of_articles.append(Type)
    new_articles = pd.DataFrame()
    new_articles["Title"]=Article_title
    new_articles["abstract"]=Article_Abstract
    new_articles["link to full article"]=Link_full_articles
    new_articles['Type']=Type_of_articles
    new_articles["pub_med_links"]=Pub_med_link
    return(new_articles)

# **Pre-processing - Sentence Tokenisation**: The following cell contains a method to tokenize abstracts into sentences.

In [ ]:
#chunk into Sentences

def tokenize_abs(df):
  tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
  ids = []
  sents = []
  for x in range(len(df)):
      data = df.iloc[x]['abstract']
      sentences = tokenizer.tokenize(data)
      for sent in sentences:
          sents.append(sent)
          ids.append(x)
  sentences_df = pd.DataFrame()
  sentences_df['id']=ids
  sentences_df['Sentence']=sents
  return(sentences_df)

In [ ]:
def filter_sentences_genes(df,list_of_genes):

  contains_ = []
  gene = []
  for x in range(len(df)):
    c = 0
    g = ''
    sent = df['Sentence'].iloc[x]
    for y in range(len(list_of_genes)):
      if (list_of_genes[y] in sent):
        c = 1
        g = list_of_genes[y]
        continue
    contains_.append(c)
    gene.append(g)
  df['Contain']=contains_
  df['Gene'] = gene
  return(df)

# **Pre-processing - Sentence Eliminator 1:** The following cell contains the method that eliminate sentences unrelated to gene regulation or expression. The method uses our pretrained classifier sentence_extraction_f.pth

In [ ]:
#Sentence classifier 1
def sentence_classification_1(df, max_len):
  model.load_state_dict(torch.load('drive/MyDrive/E. Coli/sentence_extraction_f.pth'))
  model.eval()
  Sentences = df['Sentence']
  tokenizer = BertTokenizer.from_pretrained('biobert_v1.1_pubmed', do_lower_case=True)

  tokenized_texts = list(map(lambda t: ['[CLS]']+tokenizer.tokenize(t)+['[SEP]'] , Sentences))
  #print ("Tokenize the first sentence:")
  max_len = max_len
  input_ids = pad_sequences(list(map(tokenizer.convert_tokens_to_ids, tokenized_texts)),
                            maxlen=max_len, dtype="long", truncating="post", padding="post")
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
  input_ids = pad_sequences(input_ids, maxlen=max_len, dtype="long", truncating="post", padding="post")
  attention_masks = []

  for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)
  batch_size = 8
  validation_inputs = torch.tensor(input_ids)
  validation_masks = torch.tensor(attention_masks, dtype=torch.long)

  # Create an iterator of our data with torch DataLoader

  validation_data = TensorDataset(validation_inputs, validation_masks)
  validation_sampler = SequentialSampler(validation_data)
  validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)
  all_predictions=[]
  all_predictions_0 = []
  all_predictions_1= []
  #true_labels = []
  logits_abc = []
  for batch in validation_dataloader:
      # Add batch to GPU
      batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask = batch
      # Telling the model not to compute or store gradients, saving memory and speeding up validation
      with torch.no_grad():
        # Forward pass, calculate logit predictions
        logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        #label_ids = b_labels.to('cpu').numpy()
        predicted_class = []
        logits_abc.append(logits)
        for x in range(len(logits)):
          if(logits[x][0]>0):
            abc=0
          else:
            abc=1
          all_predictions.append(abc)
        #tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        logits = logits.cpu()
        logits = (tf.nn.sigmoid(logits).numpy())
        #logits = logits.detach().cpu().numpy()
        for x in range(len(logits)):
          all_predictions_0.append(logits[x][0])
          all_predictions_1.append(logits[x][1])
          #true_labels.append(label_ids[x])
  df['predictied']=all_predictions
  df['prediction_0']=all_predictions_0
  df['prediction_1']=all_predictions_1
  print("Sentence Eliminator 1: eliminated", len(df)-np.sum(df['predictied']), "out of ",len(df))
  df = df.fillna('')
  return(df)

# **Pre-processing: Sentence Eliminator 2** The following cell contains the method that eliminate sentences that do not mention any gene/protein entities. The method uses an NER pretrained model BERN2.0.

BERN2.0 web service call may disconnect at times due to exceeding connection calls. We recommend running the program in small batches for bigger datasets.


In [ ]:
#Sentence classifier 2


def return_genes(sentence,X):
    genes = []

    for x in range(len(X['annotations'])):
        typ = X['annotations'][x]['obj']
        if(typ == "gene"):
            begin = X['annotations'][x]['span']['begin']
            end = X['annotations'][x]['span']['end']
            sent = sentence
            genes.append(sent[begin:end])
    return(genes)

def if_contains_genes(df):
  target_gene = []
  all_genes = []
  for pos in range(len(df)):

      RET = query_raw(df['Sentence'].iloc[pos])
      all_genes.append(return_genes(df['Sentence'].iloc[pos],RET))
      #print(pos,df['Sentence'].iloc[pos])
      objs = [RET['annotations'][x]['obj'] for x in range(len(RET['annotations']))].count('gene')
      if(objs>1):
          target_gene.append(1)
      else:
          target_gene.append(0)
  df['gene_mention']=target_gene
  df['All_X']= all_genes
  print("Sentence Eliminator 2: eliminated", len(df)-np.sum(df['gene_mention']), "out of ",len(df))
  df = df.fillna('')
  return(df)

# **Relation Extraction: Named entity recognition and entity tagging** The following cell contains the NER method to identify the gene/protein entities and tag sentences with all possible pairs.


In [ ]:
#NER

def return_gene_pairs(sentence,X):
    genes = []

    for x in range(len(X['annotations'])):
        typ = X['annotations'][x]['obj']
        if(typ == "gene"):
            begin = X['annotations'][x]['span']['begin']
            end = X['annotations'][x]['span']['end']
            sent = sentence
            genes.append(sent[begin:end])

    gene_pairs = []
    for row in genes:
        for column in genes:
            gene_pairs.append([row,column])

    return(gene_pairs)

def return_gene_pos(sentence,X):
    pos = []

    for x in range(len(X['annotations'])):
        typ = X['annotations'][x]['obj']
        if(typ == "gene"):
            begin = X['annotations'][x]['span']['begin']
            end = X['annotations'][x]['span']['end']
            sent = sentence
            pos.append([begin,end])
    pos = check_if_operon(sentence,pos)
    return(pos)





def return_gene_pos_pairs(sentence,X):
    pos = []

    for x in range(len(X['annotations'])):
        typ = X['annotations'][x]['obj']
        if(typ == "gene"):
            begin = X['annotations'][x]['span']['begin']
            end = X['annotations'][x]['span']['end']
            sent = sentence
            pos.append([begin,end])
    pos = check_if_operon(sentence,pos)
    gene_pos_pairs = []
    for row in pos:
        for column in pos:
            gene_pos_pairs.append([row,column])
    return(gene_pos_pairs)


def query_raw(text, url="http://bern2.korea.ac.kr/plain"):
  try:
    return requests.post(url, json={'text': text}).json()
  except:
    query_raw(text)



def check_if_operon(sentence,pos):
    new_list = pos.copy()
    pos_to_remove = []
    for x in range(len(pos)-1):

        #print(len(pos))
        #print(x,'----------------------')
        this = pos[x][1]
        next = pos[x+1][0]
        if(next-this==1):
            if(sentence[next-1]=="-"):
                pos_to_remove.append(x)
                pos_to_remove.append(x+1)
                #new_list.remove(pos[x])
                #new_list.remove(pos[x+1])
                #new_list.insert(x,[pos[x][0],pos[x+1][1]])
    #print(new_list)
    #print(pos)
    #print(pos_to_remove)
    if(len(pos_to_remove)>0):
        pos_to_remove = list(dict.fromkeys(pos_to_remove))
        pair_start = pos_to_remove[0]
        pair_end = 0
        #print(pos_to_remove)
        for y in pos_to_remove:
            new_list.remove(pos[y])


        for y in range(len(pos_to_remove)):

            if(y != len(pos_to_remove)-1):
                #print(pos_to_remove[y],pos_to_remove[y+1],pos_to_remove[y]-pos_to_remove[y+1])
                if(pos_to_remove[y]-pos_to_remove[y+1] == -1):
                    pair_end = 0
                    #print("here")
                else:
                    pair_end = pos_to_remove[y]
                if(pair_end != 0):
                    new_list.insert(pair_start,[pos[pair_start][0],pos[pair_end][1]])
                    pair_end = 0
                    pair_start = pos_to_remove[y+1]
            else:
                pair_end = pos_to_remove[y]
                new_list.insert(pair_start,[pos[pair_start][0],pos[pair_end][1]])

    return(new_list)


def NER_1(df):
  #print(len(df))
  ID = []
  sent_tagged = []
  sent_original = []
  pair = []
  add_lengths = 0
  agents=[]
  targets=[]
  All_X = []
  all_non_interested_genes = []
  new_sents = []
  sentences = list(df["Sentence"].unique())
  IDs = []
  ind = []
  for sentence in sentences:
    index = list(df[df['Sentence']==sentence]["id"])
    IDs.append(index[0])
  for x in range(len(sentences)):
      X = query_raw(sentences[x])
      #print("main")
      #print(sentences[x])
      pos = return_gene_pos_pairs(sentences[x],X)
      for j in range(len(pos)):
          #print("------------------------------")
          #print("j here",j)
          #print("pos here",len(pos))
          genes = return_gene_pairs(sentences[x],X)
          sent = sentences[x]
          if(pos[j][0] != pos[j][1]):
              agent = genes[j][0]
              target = genes[j][1]
              agent_placeholder = "AGENTAGENTAGENTAGENTAGENTAGENTAGENTAGENTAGENTAGENT"[0:len(agent)]
              target_placeholder = "TARGETTARGETTARGETTARGETTARGETTARGETTARGETTARGETTARGET"[0:len(target)]
              sent = sent[0:pos[j][0][0]]+agent_placeholder+sent[pos[j][0][1]:len(sent)]
              sent = sent[0:pos[j][1][0]]+target_placeholder+sent[pos[j][1][1]:len(sent)]
              sent = sent.replace(agent_placeholder,"GENE1").replace(target_placeholder,"GENE2")
              #print(sentences[x])
              #print(sent)
              #print("--------------------")
          else:
              #print("skipped")
              continue
              #sent = sent[0:pos[j][0][0]]+"SELFREGULATION"+sent[pos[j][0][1]:len(sent)]
              #print(sentences[x])
              #print(sent)
              #print("--------------------")
          ID.append(IDs[x])
          All_X.append(X)
          #print("j",j)
          agents.append(genes[j][0])
          targets.append(genes[j][1])
          sent_tagged.append(sent)
          #print("step1")
          #print(sent)
          sent_original.append(sentences[x])
          ind.append(x)
          genes = return_genes(sentences[x],X)
          pos_2 = return_gene_pos(sentences[x],X)
          all_non_interested_genes.append(genes)
          cur_sent = sent
          temp_sent = cur_sent
          placeholders = []
          for y in range(len(genes)):
              if("GENE1" not in genes[y] and "GENE2" not in genes[y]):
                  #print('here')
                  item = sent[pos_2[y][0]:pos_2[y][1]]
                  length = len(item)
                  placeholder = ""
                  for l in range(length):
                      placeholder = placeholder + str(y)
                  if(len(placeholder)>0):
                      placeholders.append(placeholder)

                  temp_sent = temp_sent.replace(genes[y], placeholder)
          #print(temp_sent)
          temp_sent = temp_sent.replace("GENE1","$GENE_AGENT#").replace("GENE2","$GENE_TARGET#")
          #print(placeholders)
          for p in placeholders:
              temp_sent = temp_sent.replace(p,"BLANK")
          new_sents.append(temp_sent)
          #print("------------------------------------")

          #print(x,"x------------------------------------")
          #print("step2")
          #print(temp_sent)
  new_df = pd.DataFrame()
  new_df["ID"]=ID
  new_df['ind']=ind
  new_df['Sent original'] = sent_original
  new_df['Sent tagged'] = sent_tagged
  new_df['Agent'] = agents
  new_df['Target'] = targets
  new_df["All_X"] =All_X
  new_df["all_non_interested_genes"]=all_non_interested_genes
  new_df["new_sents_blanks"]=new_sents
  new_df = new_df.fillna('')
  return(new_df)

# **Relation extraction- Relation classification:** The following cell contains the method for relation extraction. The model uses our fine-tunned pretrained BioBERT classifier relation_extraction_f.pth.


In [ ]:
#RE
def relation_extraction(df):
  model.load_state_dict(torch.load(F"drive/MyDrive/E. Coli/relation_extraction_f.pth" ))
  model.cuda()
  df_features = df[['new_sents_blanks']]
  feature = df_features['new_sents_blanks']

  tokenizer = BertTokenizer.from_pretrained('biobert_v1.1_pubmed', do_lower_case=True)

  tokenized_texts = list(map(lambda t: ['[CLS]']+tokenizer.tokenize(t)+['[SEP]'] , feature))
  #print ("Tokenize the first sentence:")
  #print (tokenized_texts[1])
  max_len = 256

  input_ids = pad_sequences(list(map(tokenizer.convert_tokens_to_ids, tokenized_texts)),
                              maxlen=max_len, dtype="long", truncating="post", padding="post")
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
  input_ids = pad_sequences(input_ids, maxlen=max_len, dtype="long", truncating="post", padding="post")
  attention_masks = []

  for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

  batch_size = 8
  validation_inputs = torch.tensor(input_ids)
  validation_masks = torch.tensor(attention_masks, dtype=torch.long)

  # Create an iterator of our data with torch DataLoader

  validation_data = TensorDataset(validation_inputs, validation_masks)
  validation_sampler = SequentialSampler(validation_data)
  validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)
  all_predictions=[]
  all_predictions_0 = []
  all_predictions_1= []
  all_predictions=[]
  #true_labels = []
  logits_abc = []
  for batch in validation_dataloader:
      # Add batch to GPU
      batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask = batch
      # Telling the model not to compute or store gradients, saving memory and speeding up validation
      with torch.no_grad():
        # Forward pass, calculate logit predictions
        logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        #label_ids = b_labels.to('cpu').numpy()
        predicted_class = []
        logits_abc.append(logits)
        for x in range(len(logits)):
          if(logits[x][0]>0):
            abc=0
          else:
            abc=1
          all_predictions.append(abc)
        #tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        logits = logits.cpu()
        logits = (tf.nn.sigmoid(logits).numpy())
        #logits = logits.detach().cpu().numpy()
        for x in range(len(logits)):
          all_predictions_0.append(logits[x][0])
          all_predictions_1.append(logits[x][1])
          #all_predictions.append(abc)

  df['prediction_RE']=all_predictions
  df['prediction_RE_0']=all_predictions_0
  df['prediction_RE_1']=all_predictions_1
  df = df.fillna('')
  return(df)

# **Post-processing- Refinement:** The following cell contains the method for relation extraction. The model uses our fine-tunned pretrained BioBERT classifier relation_extraction_f.pth.


In [ ]:
#remove Unwanted words
def remove_unwanted_wds(df):
  df_uw_2 = pd.read_excel("/content/unwanted_words.xlsx")
  agent_refined = []
  for x in range(len(df)):
      wrd = df['Agent'].iloc[x]
      if(len(df['Agent'].iloc[x].split(" "))!=1):
          wrd = ""
          for y in df['Agent'].iloc[x].split(" "):
              if(y not in list(df_uw_2['Unwanted_words'])):
                wrd =wrd + y + " "
      wrd = wrd.strip()
      agent_refined.append(wrd)
  target_refined = []
  for x in range(len(df)):
      wrd = df['Target'].iloc[x]
      if(len(df['Target'].iloc[x].split(" "))!=1):
          wrd = ""
          for y in df['Target'].iloc[x].split(" "):
              if(y not in list(df_uw_2['Unwanted_words'])):
                wrd =wrd + y + " "
      wrd = wrd.strip()
      target_refined.append(wrd)
  df['Agent_uws']=agent_refined
  df['Target_uws']=target_refined
  df = df.fillna('')
  return(df)

#remove non-gene words
def remove_non_gene(df):
  Refined_Agents = []
  for y in range(len(df)):
      #print('a',y)
      temp = df['Agent_uws'].iloc[y]
      #temp = temp.replace("-"," ")
      temp = temp.split(" ")
      gene_name = ''
      if(len(temp)>1):
          for x in range(len(temp)):
              if(len(temp[x])>0 and temp[x]!=" "):
                  X = query_raw(temp[x])
                  if('annotations' in list(X.keys())):
                    if(len(X['annotations'])>0):
                        if (X['annotations'][0]['obj'] == "gene"):
                            gene_name = gene_name + temp[x] + " "
          gene_name = gene_name.strip()
          Refined_Agents.append(gene_name)
      else:
          Refined_Agents.append(temp[0])
  Refined_Target = []
  for y in range(len(df)):
      #print('a',y)
      temp = df['Target_uws'].iloc[y]
      #temp = temp.replace("-"," ")
      temp = temp.split(" ")
      gene_name = ''
      if(len(temp)>1):
          for x in range(len(temp)):
              if(len(temp[x])>0 and temp[x]!=" "):
                  X = query_raw(temp[x])
                  if('annotations' in list(X.keys())):
                    if(len(X['annotations'])>0):
                        if (X['annotations'][0]['obj'] == "gene"):
                            gene_name = gene_name + temp[x] + " "
          gene_name = gene_name.strip()
          Refined_Target.append(gene_name)
      else:
          Refined_Target.append(temp[0])
  df['Agent_uws_gn']=Refined_Agents
  df['Target_uws_gn']=Refined_Target
  df = df.fillna('')
  return(df)

# **Post-processing: Confidence computation:** The following cell contains the methods for computing confidence factor for extracted relations.

In [ ]:
#Confidence

def compute_confidence(df2):
  confidence = []
  unique_sentences = []
  for pair in df2['Relation']:
      score = 0
      abc = df2[df2['Relation']==pair][['ID','Sent original','prediction_RE_1']]
      unique_sentences.append(len(abc["Sent original"].unique()))
      for x in abc["Sent original"].unique():
          score = score + np.average(abc[abc['Sent original']==x]['prediction_RE_1'])
      confidence.append(score)
  df2['sentence_level_c'] = confidence
  df2['re_unique_sentences']=unique_sentences #sentences giving the same relationship
  return df2

#following methods for only E Coli relations

def if_e_coli_reg(df2):
  df = pd.read_excel("/content/E_Coli_RegulonDB_regs.xlsx")
  rel_exists = []
  for x in range(len(df2)):
      agent = df2['Agent_uws_gn'].iloc[x]
      target = df2['Target_uws_gn'].iloc[x]
      rele = 0
      for y in range(len(df)):
          if(df['Agent'].iloc[y].lower() in agent.lower()):
              if(df['Target'].iloc[y].lower() in target.lower()):
                  rele = 1
                  break

      rel_exists.append(rele)
  df2['E_Coli_regulation']=rel_exists
  return df2

def if_e_coli_regulon(df2):
  df = pd.read_excel("/content/E_Coli_RegulonDB_regs.xlsx")
  E_Coli_regulons = []
  for x in list(df['Agent'].unique()):
      E_Coli_regulons.append(x.lower())
  is_regulon = []
  for agent in df2['Agent_uws_gn']:
      is_r = 0
      for ag in E_Coli_regulons:
          if(ag.lower() in agent.lower().split(" ")):
              is_r=1

              break
      is_regulon.append(is_r)
  df2['is_regulon']=is_regulon
  return df2

def final_conf(df2):
  df2['c'] = df2['sentence_level_c'] + if_e_coli_regulon(df2)['is_regulon'] + 5*if_e_coli_reg(df2)['E_Coli_regulation']
  return df2

In [ ]:
df_genes = pd.read_excel('/content/Ecoli_10_5.xlsx')
df_final = pd.DataFrame()
flag=0
for x in df_genes['Genes']:
  print("----------------------")
  #print(x)
  Keywords = "E Coli Escherichia coli gene regulation "+x
  #print(Keywords)
  Article_count = 1000
  max_len = 256
  E_Coli = search(Keywords,Article_count)

  E_Coli = search(Keywords,Article_count)

  df_articles = extract_articles(E_Coli['IdList'])
  df_articles = df_articles.dropna(axis=0)

  df_sents_1 = tokenize_abs(df_articles)
  if(len(df_sents_1)==0):

      print("here1")
      continue
  df_genes_1 = filter_sentences_genes(df_sents_1,list(df_genes['Genes']))
  #print(df_genes_1)
  df_genes_1 = df_genes_1.fillna('')
  print(x)
  print(df_genes_1['Gene'].value_counts())
  for abc in list(df_genes_1['Gene'].value_counts().index[1:len(df_genes_1['Gene'].value_counts().index)]):
    print(abc)
    print(df_genes_1['Gene'].value_counts()[abc])
    df333[x][df333['Genes']==abc]=df_genes_1['Gene'].value_counts()[abc]

----------------------
glnP
        34
glnP     7
Name: Gene, dtype: int64
glnP
7
----------------------


<ipython-input-13-7afec3fd9b71>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df333[x][df333['Genes']==abc]=df_genes_1['Gene'].value_counts()[abc]


ihfB
        52
ihfB    17
Name: Gene, dtype: int64
ihfB
17
----------------------


<ipython-input-13-7afec3fd9b71>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df333[x][df333['Genes']==abc]=df_genes_1['Gene'].value_counts()[abc]


here1
----------------------
pspG
        94
pspG     8
Name: Gene, dtype: int64
pspG
8
----------------------


<ipython-input-13-7afec3fd9b71>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df333[x][df333['Genes']==abc]=df_genes_1['Gene'].value_counts()[abc]


rtcA
    50
Name: Gene, dtype: int64
----------------------
here1
----------------------
yiaJ
        35
yiaJ     3
Name: Gene, dtype: int64
yiaJ
3
----------------------


<ipython-input-13-7afec3fd9b71>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df333[x][df333['Genes']==abc]=df_genes_1['Gene'].value_counts()[abc]


yiaM
    6981
Name: Gene, dtype: int64
----------------------
here1
----------------------
yiaO
yiaM    3
        2
Name: Gene, dtype: int64

2


<ipython-input-13-7afec3fd9b71>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df333[x][df333['Genes']==abc]=df_genes_1['Gene'].value_counts()[abc]


In [ ]:
for abc in list(df_genes_1['Gene'].value_counts().index[1:len(df_genes_1['Gene'].value_counts().index)]):
  print(abc)
  print(df_genes_1['Gene'].value_counts()[abc])
  df333[x][df333['Genes']==abc]=df_genes_1['Gene'].value_counts()[abc]


2


KeyError: ignored

In [ ]:
#df333 = pd.read_excel("/content/Ecoli_10_5_p.xlsx")
#df333.to_excel("/content/Ecoli_10_5_pk.xlsx")
#pd.read_excel('/content/Ecoli_1_50.xlsx')
df333

,Genes,glnP,ihfB,lyxK,pspG,rtcA,sgbH,yiaJ,yiaM,yiaN,yiaO
0,glnP,7,0,0,0,0,0,0,0,0,0
1,ihfB,0,17,0,0,0,0,0,0,0,0
2,lyxK,0,0,0,0,0,0,0,0,0,0
3,pspG,0,0,0,8,0,0,0,0,0,0
4,rtcA,0,0,0,0,0,0,0,0,0,0
5,sgbH,0,0,0,0,0,0,0,0,0,0
6,yiaJ,0,0,0,0,0,0,3,0,0,0
7,yiaM,0,0,0,0,0,0,0,0,0,0
8,yiaN,0,0,0,0,0,0,0,0,0,0
9,yiaO,0,0,0,0,0,0,0,0,0,0


**Step 7:** Initialize the selected keywords, number of abstracts you wish to fetch and token length for the classifiers.

In [ ]:
df_genes = pd.read_excel('/content/Ecoli1.xlsx')
df_final = pd.DataFrame()
flag=0
for x in df_genes['gene']:
  print("----------------------")
  #print(x)
  Keywords = "E Coli Escherichia coli gene regulation "+x
  #print(Keywords)
  Article_count = 1000
  max_len = 256
  E_Coli = search(Keywords,Article_count)

  E_Coli = search(Keywords,Article_count)

  df_articles = extract_articles(E_Coli['IdList'])
  df_articles = df_articles.dropna(axis=0)

  df_sents_1 = tokenize_abs(df_articles)
  if(len(df_sents_1)==0):

      print("here1")
      continue
  df_genes_1 = filter_sentences_genes(df_sents_1,list(df_genes['Genes']))
  #print(df_genes_1)
  df_genes_1 = df_genes_1.fillna('')
  print(df_genes_1['Gene'].value_counts())
  #df_sents_2 = sentence_classification_1(df_genes_1[df_genes_1['Contain']==1], max_len)
  ##if(len(df_sents_2)==0):

     #continue
  #df_sents_2 = df_sents_2.fillna('')
  #df_sents_1 = df_sents_1.fillna('')
  #df_sents_1 = if_contains_genes(df_sents_1[df_sents_1['Contain']==1])
  #df_sents_2 = sentence_classification_1(df_genes_1[df_genes_1['Contain']==1], max_len)
  df_sents_NER = NER_1(df_genes_1[df_genes_1['Contain']==1])
  if(len( df_sents_NER)==0):

    print("here3")
    continue

  df_sents_RE = relation_extraction(df_sents_NER)
  df_RE_refined_1 = remove_unwanted_wds(df_sents_RE[df_sents_RE['prediction_RE']==1])
  df_RE_refined_1 = df_RE_refined_1.fillna('')
  if(len(df_RE_refined_1)==0):

    print("here5")
    continue
  df_RE_refined_2 = remove_non_gene(df_RE_refined_1)
  df_RE_refined_2 = df_RE_refined_2.fillna('')
  if(len(df_RE_refined_2)==0):

    print("here4")
    continue
  #df_RE_refined_2 = df_RE_refined_1
  df_RE_refined_2 = df_RE_refined_2[df_RE_refined_2['Agent_uws_gn']!=""]
  df_RE_refined_2 = df_RE_refined_2[df_RE_refined_2['Target_uws_gn']!=""]
  df_RE_refined_2['Relation'] = df_RE_refined_2['Agent_uws_gn']+"--"+df_RE_refined_2['Target_uws_gn']

  df_RE_scored = compute_confidence(df_RE_refined_2)
  #for E Coli Regs
  df_RE_scored_EColi = final_conf(df_RE_scored)
  Final_relations = df_RE_scored_EColi[['Agent_uws_gn','Target_uws_gn','Relation','sentence_level_c','re_unique_sentences',	'is_regulon',	'E_Coli_regulation',	'c']].drop_duplicates()
  #--------------
  print("Final Relations extracted as as follows")
  Final_relations.sort_values(['c'],ascending=False).to_excel("/content/drive/MyDrive/E. Coli/ExGI_result_10_5"+str(x)+".xlsx")
  Final_relations.sort_values(['c'],ascending=False)
  if flag==0:
    flag=1
    df_final=Final_relations
  else:
    df_final = df_final.append(Final_relations, ignore_index=True)
  print('Final_relations',len(Final_relations))
  Final_relations.to_excel('drive/MyDrive/E. Coli/Ecoli_50_1_'+str(x)+'.xlsx')
  df_final.to_excel('drive/MyDrive/E. Coli/Ecoli_50_1_df_final'+str(x)+'.xlsx')
  print('df_final',len(df_final))

----------------------


KeyError: ignored

In [ ]:
pd.read_excel('/content/Ecoli1.xlsx')

,gene,cho,ddlB,dinB,dinD,dinF,dinG,dinI,dinJ,dinQ,...,uvrC,uvrD,uvrY,yafN,yafO,yafP,yafQ,ybfE,ydjM,yebG
0,cho,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ddlB,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,dinB,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,dinD,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,dinF,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,dinG,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,dinI,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,dinJ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,dinQ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,dnaG,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
Final_relations  = xxx
ag_y = []
tr_y = []
for x in range(len(Final_relations)):
    tempa = 0
    for y in list(df_genes['Genes']):
        if(y in Final_relations["Agent_uws_gn"].iloc[x]):
            tempa= 1
            #ag_y.append(tempa)
            continue
    ag_y.append(tempa)
    tempb = 0
    for y in list(df_genes['Genes']):
        if(y in Final_relations["Target_uws_gn"].iloc[x]):
            tempb= 1
            #ag_y.append(tempa)
            continue
    tr_y.append(tempb)
Final_relations['ag_y']=ag_y
Final_relations['tr_y']=tr_y

In [ ]:
df_RE_refined_2 = df_RE_refined_2[df_RE_refined_2['Agent_uws_gn']!=""]
df_RE_refined_2 = df_RE_refined_2[df_RE_refined_2['Target_uws_gn']!=""]
df_RE_refined_2['Relation'] = df_RE_refined_2['Agent_uws_gn']+"--"+df_RE_refined_2['Target_uws_gn']

df_RE_scored = compute_confidence(df_RE_refined_2)
  #for E Coli Regs
df_RE_scored_EColi = final_conf(df_RE_scored)
Final_relations = df_RE_scored_EColi[['Agent_uws_gn','Target_uws_gn','Relation','sentence_level_c','re_unique_sentences',	'is_regulon',	'E_Coli_regulation',	'c']].drop_duplicates()
  #--------------
print("Final Relations extracted as as follows")
Final_relations.sort_values(['c'],ascending=False).to_excel("/content/drive/MyDrive/E. Coli/ExGI_result"+str(x)+".xlsx")
Final_relations.sort_values(['c'],ascending=False)
if flag==0:
  flag=1
  df_final=Final_relations
else:
  df_final = df_final.append(Final_relations, ignore_index=True)
  print('Final_relations',len(Final_relations))
  Final_relations.to_excel('drive/MyDrive/E. Coli/Ecoli_3_'+str(x)+'.xlsx')
  df_final.to_excel('drive/MyDrive/E. Coli/Ecoli_3_df_final'+str(x)+'.xlsx')
  print('df_final',len(df_final))

Final Relations extracted as as follows


In [ ]:
Final_relations.to_excel("xxx.xlsx")

In [ ]:
df_articles.to_excel("drive/MyDrive/E. Coli/e_coli_2_articles.xlsx")
df_final.to_excel("drive/MyDrive/E. Coli/e_coli_2_rels.xlsx")

In [ ]:
Keywords = "E Coli Escherichia coli gene regulation gene expression transcriptional"
Article_count = 1000
max_len = 256
Keywords = "gene regulation transcriptional Bacillus subtilis"
E_Coli = search(Keywords,Article_count)
#df_genes = pd.read_excel('/content/SOS_DNA_9Genes_Network.xlsx')
#E_Coli = search(Keywords,Article_count)
df_articles = extract_articles(E_Coli['IdList'])
df_articles = df_articles.dropna(axis=0)
df_sents_1 = tokenize_abs(df_articles)

In [ ]:
#df_genes_1['Gene'].value_counts()
#np.sum(df_genes_1['Contain'])
#len(df_articles)
df_genes = pd.read_excel('/content/SOS_DNA_9Genes_Network.xlsx')

In [ ]:
df_RE_refined_2['Relation'] = df_RE_refined_2['Agent_uws_gn']+"--"+df_RE_refined_2['Target_uws_gn']

UFuncTypeError: ignored

**Step 8:** Run the following cell to extract the relations

In [ ]:
df_genes = pd.read_excel('/content/Ecoli_1.xlsx')
E_Coli = search(Keywords,Article_count)

df_articles = extract_articles(E_Coli['IdList'])
df_articles = df_articles.dropna(axis=0)
df_sents_1 = tokenize_abs(df_articles)

df_genes_1 = filter_sentences_genes(df_sents_all,list(df_genes['Genes']))

print(df_genes_1['Gene'].value_counts())
df_sents_2 = sentence_classification_1(df_genes_1[df_genes_1['Contain']==1], max_len)
df_genes_1 = df_genes_1.fillna('')
b = 6
abc = int(len(df_genes_1[df_genes_1['Contain']==1])/b)

df_sents_3 = pd.DataFrame()
for x in range(b):
  df_temp = df_genes_1[df_genes_1['Contain']==1][x*abc: (x+1)*abc]
  #print(x,abc)
  df_sents_3_t = if_contains_genes(df_genes_1[df_genes_1['Contain']==1])
  if(x == 0 ):
    df_sents_3 = df_sents_3_t
  else:
    df_sents_3 = df_sents_3.append(df_sents_3_t, ignore_index=True)
df_sents_3 = df_sents_3.fillna('')
df_sents_NER = NER_1(df_sents_3[df_sents_3["gene_mention"]==1])
df_sents_RE = relation_extraction(df_sents_NER)
df_RE_refined_1 = remove_unwanted_wds(df_sents_RE[df_sents_RE['prediction_RE']==1])
df_RE_refined_1 = df_RE_refined_1.fillna('')
df_RE_refined_2 = remove_non_gene(df_RE_refined_1)
df_RE_refined_2 = df_RE_refined_2.fillna('')
df_RE_refined_2 = df_RE_refined_2[df_RE_refined_2['Agent_uws_gn']!=""]
df_RE_refined_2 = df_RE_refined_2[df_RE_refined_2['Target_uws_gn']!=""]
df_RE_refined_2['Relation'] = df_RE_refined_2['Agent_uws_gn']+"--"+df_RE_refined_2['Target_uws_gn']
df_RE_scored = compute_confidence(df_RE_refined_2)
#for E Coli Regs
df_RE_scored_EColi = final_conf(df_RE_scored)
Final_relations = df_RE_scored_EColi[['Agent_uws_gn','Target_uws_gn','Relation','sentence_level_c','re_unique_sentences',	'is_regulon',	'E_Coli_regulation',	'c']].drop_duplicates()
#--------------
print("Final Relations extracted as as follows")
print(Final_relations)
Final_relations.sort_values(['c'],ascending=False).to_excel("ExGI_result.xlsx")
Final_relations.sort_values(['c'],ascending=False)

FileNotFoundError: ignored

In [ ]:
#df_sents_3 = df_sents_3.fillna('')
#df_sents_NER_1 = NER_1(df_sents_3[df_sents_3["gene_mention"]==1].iloc[0:18])
#df_sents_NER_2 = NER_1(df_sents_3[df_sents_3["gene_mention"]==1])
#df_sents_RE = relation_extraction(df_sents_NER_2)
#df_RE_refined_1 = remove_unwanted_wds(df_sents_RE[df_sents_RE['prediction_RE']==1])
#df_RE_refined_1 = df_RE_refined_1.fillna('')
df_RE_refined_2 = remove_non_gene(df_RE_refined_1)
df_RE_refined_2 = df_RE_refined_2.fillna('')
df_RE_refined_2 = df_RE_refined_2[df_RE_refined_2['Agent_uws_gn']!=""]
df_RE_refined_2 = df_RE_refined_2[df_RE_refined_2['Target_uws_gn']!=""]
df_RE_refined_2['Relation'] = df_RE_refined_2['Agent_uws_gn']+"--"+df_RE_refined_2['Target_uws_gn']
df_RE_scored = compute_confidence(df_RE_refined_2)
#for E Coli Regs
df_RE_scored_EColi = final_conf(df_RE_scored)
Final_relations = df_RE_scored_EColi[['Agent_uws_gn','Target_uws_gn','Relation','sentence_level_c','re_unique_sentences',	'is_regulon',	'E_Coli_regulation',	'c']].drop_duplicates()
#--------------
print("Final Relations extracted as as follows")
Final_relations.sort_values(['c'],ascending=False).to_excel("ExGI_result.xlsx")
Final_relations.sort_values(['c'],ascending=False)

Final Relations extracted as as follows


,Agent_uws_gn,Target_uws_gn,Relation,sentence_level_c,re_unique_sentences,is_regulon,E_Coli_regulation,c
58,LexA,SOS,LexA--SOS,14.298716,15,1,0,15.298716
63,SOS,LexA,SOS--LexA,13.787022,14,0,0,13.787022
89,LexA,RecA,LexA--RecA,6.913495,7,1,1,12.913495
148,LexA,LexA,LexA--LexA,5.898354,6,1,1,11.898354
1946,lexA,recA,lexA--recA,5.794406,6,1,1,11.794406
...,...,...,...,...,...,...,...,...
2086,recA,recA430,recA--recA430,0.551123,1,0,0,0.551123
1409,SOS,ydjM,SOS--ydjM,0.537277,1,0,0,0.537277
404,recA(E38K),SOS,recA(E38K)--SOS,0.521291,1,0,0,0.521291
411,colicin,lexA,colicin--lexA,0.499577,1,0,0,0.499577


In [ ]:
Final_relations = df_final
ag_y = []
tr_y = []
for x in range(len(Final_relations)):
  if Final_relations["Agent_uws_gn"].iloc[x] in list(df_genes['Genes']):
    ag_y.append(1)
  else:
    ag_y.append(0)
  if Final_relations["Target_uws_gn"].iloc[x] in list(df_genes['Genes']):
    tr_y.append(1)
  else:
    tr_y.append(0)
Final_relations['ag_y']=ag_y
Final_relations['tr_y']=tr_y

In [ ]:
Final_relations.to_excel("E_Coli_1_final_rels.xlsx")

In [ ]:
for x in range(3,b):
  df_temp = df_genes_1[df_genes_1['Contain']==1][x*abc: (x+1)*abc]
  print(x,abc)
  df_sents_3_t = if_contains_genes(df_temp)
  if(x == 0 ):
    df_sents_3 = df_sents_3_t
  else:
    df_sents_3 = df_sents_3.append(df_sents_3_t, ignore_index=True)

3 625
Sentence Eliminator 2: eliminated 182 out of  625
4 625


<ipython-input-29-0c67cf66a394>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_sents_3 = df_sents_3.append(df_sents_3_t, ignore_index=True)


Sentence Eliminator 2: eliminated 150 out of  625
5 625


<ipython-input-29-0c67cf66a394>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_sents_3 = df_sents_3.append(df_sents_3_t, ignore_index=True)


Sentence Eliminator 2: eliminated 115 out of  625


<ipython-input-29-0c67cf66a394>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_sents_3 = df_sents_3.append(df_sents_3_t, ignore_index=True)


In [ ]:
len(df_sents_3)/625

3.0

In [ ]:
Final_relations.sort_values(['c'],ascending=False).to_excel("ExGI_result.xlsx")

In [ ]:
def NER_1(df):
  #print(len(df))
  ID = []
  sent_tagged = []
  sent_original = []
  pair = []
  add_lengths = 0
  agents=[]
  targets=[]
  All_X = []
  all_non_interested_genes = []
  new_sents = []
  sentences = list(df["Sentence"].unique())
  IDs = []
  ind = []
  for sentence in sentences:
    index = list(df[df['Sentence']==sentence]["id"])
    IDs.append(index[0])
  for x in range(len(sentences)):
      X = query_raw(sentences[x])
      #print(sentences[x])
      #print("main")
      #print(sentences[x])
      pos = return_gene_pos_pairs(sentences[x],X)
      for j in range(len(pos)):
          #print("------------------------------")
          #print("j here",j)
          #print("pos here",len(pos))
          genes = return_gene_pairs(sentences[x],X)
          sent = sentences[x]
          if(pos[j][0] != pos[j][1]):
              agent = genes[j][0]
              target = genes[j][1]
              agent_placeholder = "*!*!*!*!*!*!*!*!*!*!*!*!*!*!*!*!*!*!*!*!*!*!*!*!*!*!*!*!"[0:len(agent)]
              target_placeholder = "^(#^(#^(#^(#^(#^(#^(#^(#^(#^(#^(#^(#^(#^(#^(#^(#^(#^(#^(#^(#^(#"[0:len(target)]
              #print(sent)
              #print(pos[j])
              sent = sent[0:pos[j][0][0]]+agent_placeholder+sent[pos[j][0][1]:len(sent)]
              #print(sent)
              #print(pos[j])
              sent = sent[0:pos[j][1][0]]+target_placeholder+sent[pos[j][1][1]:len(sent)]
              #print(sent)
              #print(pos[j])
              sent = sent.replace(agent_placeholder,"GENE1").replace(target_placeholder,"GENE2")
              #print(sentences[x])
              #print(sent)
              #print("--------------------")
          else:
              #print("skipped")
              continue
              #sent = sent[0:pos[j][0][0]]+"SELFREGULATION"+sent[pos[j][0][1]:len(sent)]
              #print(sentences[x])
              #print(sent)
              #print("--------------------")
          ID.append(IDs[x])
          All_X.append(X)
          #print("j",j)
          agents.append(genes[j][0])
          targets.append(genes[j][1])
          sent_tagged.append(sent)
          #print("step1")
          #print(sent)
          sent_original.append(sentences[x])
          ind.append(x)
          genes = return_genes(sentences[x],X)
          pos_2 = return_gene_pos(sentences[x],X)
          all_non_interested_genes.append(genes)
          cur_sent = sent
          temp_sent = cur_sent
          placeholders = []

          for y in range(len(genes)):
              #print(genes)
              if("GENE1" not in genes[y] and "GENE2" not in genes[y]):
                  #print('here')
                  #print(genes[y])
                  #print(sent)
                  #print(pos_2)
                  try:
                    item = sent[pos_2[y][0]:pos_2[y][1]]
                    length = len(item)
                    placeholder = ""
                    for l in range(length):
                        placeholder = placeholder + str(y)
                    if(len(placeholder)>0):
                        placeholders.append(placeholder)
                  except:
                    continue


                  temp_sent = temp_sent.replace(genes[y], placeholder)
          #print(temp_sent)
          temp_sent = temp_sent.replace("GENE1","$GENE_AGENT#").replace("GENE2","$GENE_TARGET#")
          #print(placeholders)
          for p in placeholders:
              temp_sent = temp_sent.replace(p,"BLANK")
          new_sents.append(temp_sent)
          #print("------------------------------------")

          #print(x,"x------------------------------------")
          #print("step2")
          #print(temp_sent)
  new_df = pd.DataFrame()
  new_df["ID"]=ID
  new_df['ind']=ind
  new_df['Sent original'] = sent_original
  new_df['Sent tagged'] = sent_tagged
  new_df['Agent'] = agents
  new_df['Target'] = targets
  new_df["All_X"] =All_X
  new_df["all_non_interested_genes"]=all_non_interested_genes
  new_df["new_sents_blanks"]=new_sents
  new_df = new_df.fillna('')
  return(new_df)


In [ ]:
df_sents_NER_2

NameError: ignored

In [ ]:
df_genes_1 = filter_sentences_genes(df_sents_1,list(df_genes['Genes']))

In [ ]:
np.sum(filter_sentences_genes(df_genes_1,list(df_genes['Genes']))['Contain'])

362

In [ ]:
df_genes_1['Gene'].value_counts()

        1080
LexA     139
recA      96
lexA      70
uvrA      25
umuD      23
uvrD       6
polB       2
ruvA       1
Name: Gene, dtype: int64

In [ ]:
" ".join(list(df_genes['Genes']))

'LexA recA lexA umuD uvrD uvrA uvrY ruvA polB'

In [ ]:
df_sents_1[df_sents_1['Contain']==1]

,id,Sentence,Contain,Gene
28,3,The bacterial DNA damage response pathway (SOS...,1,LexA
29,3,"The lexA promoter, itself, contains two LexA o...",1,lexA
31,3,"To test this, we quantified 5mC at the lexA lo...",1,lexA
32,3,We found that 5mC at the lexA promoter is spec...,1,lexA
33,3,Our data support a model where LexA binding at...,1,LexA
...,...,...,...,...
1422,171,These data can be explained by a model in whic...,1,LexA
1424,172,A comparison of these sequences with those of ...,1,lexA
1425,172,"Using both, a thermosensitive lexA host strain...",1,lexA
1426,172,"From the data presented, we conclude that bact...",1,lexA
